In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir(globals()['_dh'][0])
os.chdir('..')
# print(os.path.abspath(os.curdir))

In [2]:
from src.torchwavenet import Wave
from src.torchdataset import RFMixtureDatasetBase
from omegaconf import OmegaConf
import torch
from torch.utils.data import DataLoader

from torchviz import make_dot

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

Using device:  cuda


In [4]:
dataset = RFMixtureDatasetBase(
    root_dir="npydataset/Dataset_QPSK_CommSignal2_Mixture")
loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [5]:
batch = next(iter(loader))
batch["sample_mix"].shape, batch["sample_soi"].shape

(torch.Size([16, 2, 40960]), torch.Size([16, 2, 40960]))

In [6]:
cfg = OmegaConf.load("src/configs/wavenet.yml")
model = Wave(cfg.model).to(device)
model

Wave(
  (input_projection): Conv1d(2, 128, kernel_size=(1,), stride=(1,))
  (residual_layers): ModuleList(
    (0): ResidualBlock(
      (dilated_conv): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (output_projection): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    )
    (1): ResidualBlock(
      (dilated_conv): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
      (output_projection): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    )
    (2): ResidualBlock(
      (dilated_conv): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
      (output_projection): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    )
    (3): ResidualBlock(
      (dilated_conv): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
      (output_projection): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    )
    (4): ResidualBlock(
      (dilated_conv): Conv1d(128, 256, kernel_size=(3,), stride=(1,)

In [7]:
str(sum(p.numel() for p in model.parameters()
    if p.requires_grad) / 1e6) + " million parameters"

'3.964674 million parameters'

In [8]:
dot = make_dot(model(batch["sample_mix"].to(device)),
               params=dict(list(model.named_parameters())))

OutOfMemoryError: CUDA out of memory. Tried to allocate 320.00 MiB. GPU 0 has a total capacity of 39.39 GiB of which 100.81 MiB is free. Process 25829 has 39.28 GiB memory in use. Of the allocated memory 38.77 GiB is allocated by PyTorch, and 19.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
for node in dot.body:
    print(node)

	140574821759120 [label="
 (16, 2, 40960)" fillcolor=darkolivegreen1]
